In [157]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd
from time import sleep
from tqdm import tqdm


creds = None
with open('../../secret.json') as creds_file:
    creds = json.load(creds_file)


# Set up the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

print('hi')
print(sp)


hi


In [158]:
def get_genres(genres):
    # first one category, second genre --- can change this if want
    if len(genres) >= 2:
        return genres[0], genres[1]
    elif len(genres) >= 1:
        return genres[0], genres[0]
    else:
        return "Data", "Data"

In [159]:
def get_album_genre(album_name, artist_name):
  
    q_album = "artist:{} album:{}".format(artist_name, album_name)
    results_album = sp.search(q_album, limit=1, offset=0, type='album')

    if len(results_album['albums']['items']) > 0:
        
        # try grabbing genre through album
        album_id = results_album['albums']['items'][0]['id']
        album = sp.album(album_id)
        genres = album['genres']
        if len(genres) > 0:
            return get_genres(genres)
        else:
            # try grabbing genre through artist
            artist_id = results_album['albums']['items'][0]['artists'][0]['id']
            artist_info = sp.artist(artist_id)
            genres = artist_info['genres']
            return get_genres(genres)
    else:
        #just try searching the artitst
        q_artist = "artist:{}".format(artist_name)
        results_artist = sp.search(q_artist, limit=1, offset=0, type='artist')
        if len(results_artist['artists']['items']) == 1:
            genres = results_artist['artists']['items'][0]['genres']
            return get_genres(genres)
        else:
            return "Data", "Data"

In [160]:
artist_name = "Linkin Park"
song_name = ""
try:
    res = get_album_genre(song_name, artist_name)
    print(res)
except Exception as e:
    print(str(e))

('alternative metal', 'nu metal')


In [161]:

cddb_df = pd.read_csv('../../cleaning_stages/4_year_repair/cddb_3.tsv', sep='\t')
tracks_df = pd.read_csv('../../cleaning_stages/4_year_repair/cddb_tracks.tsv', sep='\t')

cddb_genre_1 = cddb_df[cddb_df['genre'] == 'Data']
cddb_genre_2 = cddb_df[cddb_df['genre'].isnull()]
cddb_genre_3 = cddb_df[cddb_df['category'].isnull()]

combine = [cddb_genre_1, cddb_genre_2, cddb_genre_3]
  
cddb_trim = pd.concat(combine)
cddb_df_trim = cddb_trim[['artist', 'title', 'id']]
print(len(cddb_df_trim))
print(cddb_df_trim)

33
                             artist   
2                         No Return  \
4                           Emanuel   
14                            Keedy   
25                  Jazz Signatures   
26              Bernadã¨te Bidã ude   
33             Sportfreunde Stiller   
34          Rennie Pilgrim Presents   
38                             Deux   
40                          Various   
41                  Matsuka Phonics   
43                         Robotika   
48                   Rolling Stones   
49     Ã¤ã®ã«ã¥ã°ã¤ Ã¹ã¬ Ã¤ã¢ã¸ã¥ã¡   
61            Â¿â©ã‡ã _â½âºã„ã‰ã„â¡   
62                     Diversetion)   
9332                     Pink Floyd   
1                           Various   
10                          Various   
11             Â¾ã®â¸â°ã€ãœ Worship   
17                       Bill Evans   
18                 Nature Destroyed   
20                Dingdong Avanzado   
22    Variousce Italiennes (disc 1)   
29                           <v8.5e   
31                    

In [162]:
# lists are much faster than dataframes, converting
cddb_list = cddb_df_trim.values.tolist()
tracks_list = tracks_df.values.tolist()

print(f'num null years: ${len(cddb_list)}')

num null years: $33


In [165]:
def start_repair(cd_list):
    
    for cddb_row in cd_list:

        artist_name = cddb_row[0]
        album_name = cddb_row[1]
        category, genre = get_album_genre(album_name, artist_name)

        # update cddb_df directly
        # if no results it will just fill in "Data", "Data"
        idx_list = cddb_df.index[cddb_df['id'] == cddb_row[2]].tolist()
        idx = idx_list[0]
        cddb_df.at[idx,'category'] = category
        cddb_df.at[idx,'genre'] = genre
        print("fixed - :" + str(cddb_row) + "category: " + str(category) + ", " + "genre: " + str(genre))
    

In [166]:
start_repair(cddb_list)

fixed - :['No Return', 'Self Mutilation', 100002]category: Data, genre: Data
fixed - :['Emanuel', 'Felicidade', 100004]category: classic portuguese pop, genre: classic portuguese pop
fixed - :['Keedy', 'Chase The Clouds', 100015]category: bounce, genre: bounce
fixed - :['Jazz Signatures', 'Jazz Signatures', 100028]category: Data, genre: Data
fixed - :['Bernadã¨te Bidã\xa0ude', "Pomme D'amour", 100029]category: Data, genre: Data
fixed - :['Sportfreunde Stiller', 'Die Gute Seite', 100037]category: german indie, genre: german indie rock
fixed - :['Rennie Pilgrim Presents', 'Nu Skool Breakz 2', 100038]category: Data, genre: Data
fixed - :['Deux', 'Forever', 100042]category: dfw rap, genre: dfw rap
fixed - :['Various', 'Custom Music Cd', 100044]category: pluggnb, genre: pluggnb
fixed - :['Matsuka Phonics', "Let's Study Phonics 2", 100045]category: Data, genre: Data
fixed - :['Robotika', 'Dance', 100047]category: Data, genre: Data
fixed - :['Rolling Stones', 'Giant Stadium 17.08.1994 Cd 2', 

In [167]:
#save to output
cddb_df.to_csv('../../cleaning_stages/5_genre_repair/cddb_4.tsv', sep="\t", index=False)
tracks_df.to_csv('../../cleaning_stages/5_genre_repair/cddb_tracks.tsv', sep="\t", index=False)